In [1]:
#!pip install pystan==3.3.0

In [6]:
import pystan
pystan.__version__

'2.19.1.1'

In [8]:
%%time
import pystan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

sm = pystan.StanModel(model_code=schools_code)
fit = sm.sampling(data=schools_data, iter=1000, chains=4, seed=1)
fit.extract()["eta"]  # array with shape (2000, 8)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_67cb7d0f2cb7720776cbeb52007d2dbb NOW.


CPU times: user 1.58 s, sys: 134 ms, total: 1.71 s
Wall time: 1min 5s


In [9]:
fit

Inference for Stan model: anon_model_67cb7d0f2cb7720776cbeb52007d2dbb.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu         8.09    0.16   4.87  -1.05   4.79   8.02   11.3  18.05    910    1.0
tau        6.46     0.2   5.21   0.33   2.58   5.37   8.76  18.87    711    1.0
eta[1]     0.38    0.02   0.95  -1.49  -0.25    0.4   1.04   2.11   1866    1.0
eta[2]    -0.01    0.02    0.9  -1.76  -0.61 8.3e-3   0.58   1.75   1806    1.0
eta[3]    -0.21    0.02   0.94  -2.01  -0.84  -0.23    0.4   1.68   1822    1.0
eta[4]    -0.02    0.02   0.87  -1.81  -0.59  -0.03   0.54   1.69   1848    1.0
eta[5]    -0.39    0.02   0.88  -2.12  -0.97   -0.4   0.15   1.39   1710    1.0
eta[6]    -0.23    0.02   0.88  -1.99  -0.81  -0.21   0.34   1.49   1902    1.0
eta[7]     0.34    0.02   0.87  -1.36  -0.22   0.39   0.93   2.02   1814    1.0
eta[8]     0.